# First Steps

This is an example of what you can already with GMT/Python. 
We'll mostly reproduce some examples from the [GMT Tutorial](http://gmt.soest.hawaii.edu/doc/latest/GMT_Tutorial.html#session-one).

## Loading the library

The GMT modules are available as functions and classes in the `gmt` Python package. 
So we'll start by importing it:

In [ ]:
import gmt

## A first map

All figure generation in GMT/Python is handled by the `gmt.Figure` class. 
It has methods to add layers to your figure, like a basemap, coastlines, and data.

We start a new figure by creating an instance of `gmt.Figure`:

In [ ]:
fig = gmt.Figure()

We add elements to the figure using its methods. For example, lets add the coastlines of Central America to a 6 inch wide map using the Mercator projection (`M`). Our figure will also have a nice frame with automatic tics.

In [ ]:
fig.coast(region=[-90, -70, 0, 20], projection='M6i', land='chocolate', 
          frame=True)

You can see a preview of the figure directly in the [Jupyter notebook](http://jupyter.org) using `fig.show()`.

In [ ]:
fig.show()

To open a PDF preview of the figure using your default PDF viewer use:

```python
fig.show(external=True)
```

This is useful when using the Python shell or IPython terminal app. 
However, **this command will not interrupt your Python process**. 
So using it in a Python script will not work because the script will finish and delete the generated previews.
Use `fig.savefig` to save the figure to a file instead (see below).

## A note for experienced GMT users

You'll probably have noticed several things that are different from classic command-line GMT.
Many of these changes reflect the new GMT [modern execution mode](http://gmt.soest.hawaii.edu/projects/gmt/wiki/Modernization) that will be part of the future 6.0 release.
A few are GMT/Python exclusive (like the long argument names).

1. The name of method is `coast` instead of `pscoast`. As a general rule, all `ps*` modules had their `ps` removed. The exceptions are: `psxy == plot`, `psxyz == plot3d`, and `psscale == colorbar`.
2. The arguments don't use the GMT 1-letter syntax (R, J, B, etc). Those are still available as aliases and the methods will accept them (see below). 
3. Arguments like `region` can take lists instead of strings like `1/2/3/4`. You can still use the string form but the list form is easier in Python.
4. If a GMT argument has no options (like `-B` instead of `-Baf`), use a `True` value instead. An empty string would also be acceptable.
5. There is no output redirecting to a PostScript file. The figure is generated in the background and will only be shown or saved when you ask for it.

We could have generated the figure above using the classic GMT argument names (but not the module names):

In [ ]:
fig_alias = gmt.Figure()
fig_alias.coast(R='-90/-70/0/20', J='M6i', G='chocolate', B=True)
fig_alias.show()

## Saving the figure

Unlike the GMT command-line interface, **no figure file was generated until you ask for one**.

Use method `fig.savefig` (based on the [matplotlib](https://matplotlib.org/) function) to save your figure to a file:

In [ ]:
fig.savefig('first-steps-central-america.png')

If you're running a Python script, you can tell `fig.savefig` to open the figure in an external viewer:

```python
fig.savefig('first-steps-central-america.png', show=True)
```

The classic GMT command `psconvert` is also available (and is used by `savefig` in the background):

In [ ]:
# Format 'f' means a PDF
fig.psconvert(prefix='first-steps-central-america', fmt='f')

## Plot point data

We can use `gmt.Figure.plot` to plot data on our map.

First, lets create some fake data to plot using numpy:

In [ ]:
import numpy as np

In [ ]:
# See the random number generator so we always 
# get the same numbers
np.random.seed(42)

lon = np.random.uniform(150, 240, 30)
lat = np.random.uniform(-30, 60, 30)

Save the data to a file in 2 columns:

In [ ]:
np.savetxt('first-steps-data.txt', np.transpose([lon, lat]))

Now we can plot the data using `Figure.plot` and passing the file name as the `data` argument:

In [ ]:
fig_globe = gmt.Figure()
fig_globe.coast(
    region='g', projection='G200/30/6i', frame='ag', 
    resolution='i', area_thresh=5000, land='white', 
    water='DarkTurquoise')
fig_globe.plot(data='first-steps-data.txt', style='s0.3c',
               color='black')
fig_globe.show()

GMT ships some sample data that can be accessed by passing `@data_file` as the `data` argument. 

For example, we can plot the earthquake epicenters from the `tut_quakes.ngdc` sample dataset:

In [ ]:
fig_quakes = gmt.Figure()
fig_quakes.coast(
    region=[130, 150, 35, 50], projection='M6i', 
    frame='afg', shorelines=True, land='gray', 
    water='lightblue')
fig_quakes.plot(data='@tut_quakes.ngdc', style='c0.3c', 
                color='blue', pen='faint', columns=[4, 3])
fig_quakes.show()